In [12]:
import pandas as pd
import numpy as np

### the format xlsx for excel files need the package xlrd - installing that with "!pip install xlrd"

In [27]:
product = pd.read_excel(r"./dataset/product.xlsx")
order_data = pd.read_csv(r"./dataset/tarikhche kharid.csv")
comments = pd.read_excel(r"./dataset/comment.xlsx")
orders = pd.read_csv(r"./dataset/orders.csv")
quality = pd.read_excel(r"./dataset/keifiat.xlsx")
product.head()

,id,product_title_fa,product_title_en,url_code,title_alt,category_title_fa,category_keywords,brand_name_fa,brand_name_en,product_attributes
0,723302,ماگ حرارتی لومانا مدل عرفانه کد MAG1042,NaN,ماگ-حرارتی-لومانا-مدل-عرفانه-کد-mag1042,mug، لیوان ، ماگ ، فنجان ، لیوان دسته دار ، لو...,ماگ، لیوان و فنجان,ماگ، لیوان و فنجان CupandMug,لومانا,Lomana,"[{""Key"":""وزن بسته‌بندی"",""Value"":""480""},{""Key"":..."
1,727089,ست آبکش و آبگیر تیتیز مدل AP-9141 سایز متوسط,NaN,ست-آبکش-و-آبگیر-تیتیز-مدل-ap-9141-سایز-متوسط,";hsi, steel, کاسه، کاسه بامبو، لگن، کالا خانه ...",آبکش و آبگیر,آبکش و آبگیر VirgoBasin,تیتیز,Titiz,"[{""Key"":""دستگیره"",""Value"":""1 جفت""},{""Key"":""نوع..."
2,727096,روغن مرطوب کننده کدلی مدل Divine oil حجم 50 می...,NaN,روغن-مرطوب-کننده-کدلی-مدل-divine-oil-حجم-50-می...,NaN,روغن صورت,روغن صورت Face-Oil,کدلی,CAUDALIE,"[{""Key"":""حجم"",""Value"":""50""},{""Key"":""نوع عصاره""..."
3,727803,ست لحاف پوپلین هوبی مدل اما یکنفره چهار تکه کاوری,Hobby Emma Quilt Single Set - 4 Pieces,ست-لحاف-پوپلین-هوبی-مدل-اما-یکنفره-چهار-تکه-کاوری,روتختی اسپرت، اما، emma، روتختی یکنفره، لحاف ی...,سرویس خواب,سرویس خواب SleepSet,هوبی,Hobby,"[{""Key"":""ابعاد رو بالشی"",""Value"":""50×70""},{""Ke..."
4,725222,ماگ آبنبات رنگی مدل AR0143,NaN,ماگ-آبنبات-رنگی-مدل-ar0143,ماگ آبنبات رنگی،شذدشذشفقشدله،Mag Abnabatrangi,ماگ، لیوان و فنجان,ماگ، لیوان و فنجان CupandMug,متفرقه,Miscellaneous,"[{""Key"":""در"",""Value"":""ندارد""},{""Key"":""جنس"",""Va..."


In [24]:
perc =[.25, .50, .75] 
include =['object', 'float', 'int']

In [25]:
product.describe(percentiles=perc, include=include)

,product_title_fa,product_title_en,url_code,title_alt,category_title_fa,category_keywords,brand_name_fa,brand_name_en,product_attributes
count,100000,21165,99983,77432,100000,99740,100000,100000,3050
unique,97446,18356,97130,42396,1206,1185,2786,2819,2967
top,خراب,/,خراب,زونکن . زونکن پیکو . قیمت زونکن پیکو . پیکو . ...,کیف و کاور گوشی,کیف و کاور گوشی Cell-Phone-Pouch-Cover,متفرقه,Miscellaneous,"[{""Key"":""وزن"",""Value"":""80""},{""Key"":""سایر توضیح..."
freq,32,718,32,396,4384,4384,56478,56478,5


In [29]:
orders.describe(percentiles=perc, include=include)

,DateTime_CartFinalize,Amount_Gross_Order,city_name_fa,Quantity_item
count,200000,2.000000e+05,200000,200000.000000
unique,197293,NaN,906,NaN
top,2016-02-29 13:23:57.000,NaN,تهران,NaN
freq,4,NaN,108306,NaN
mean,NaN,1.458204e+06,NaN,1.261225
std,NaN,5.450972e+06,NaN,1.801186
min,NaN,0.000000e+00,NaN,1.000000
25%,NaN,1.212730e+05,NaN,1.000000
50%,NaN,3.211010e+05,NaN,1.000000
75%,NaN,1.011032e+06,NaN,1.000000
